## 1: Database Connection

In [ ]:
url = ""
db = ""
api_key = ""

In [ ]:
import requests

## 2 SO Creation

### 2.1: Search Partner

In [ ]:
partner = requests.post(
    f"{url}/json/2/res.partner/search",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "domain": [
            ("name", "=", "Deco Addict"),
        ],
    },
).json()
print("Partner id", partner)

In [ ]:
# Example of other queries that could have been done, with different arguments
partners = requests.post(
    f"{url}/json/2/res.partner/search",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "domain": [
            ("name", "ilike", "Deco"),
        ],
    },
).json()
print("Partner ids (without limit/offset)", partners)

partners = requests.post(
    f"{url}/json/2/res.partner/search",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "domain": [
            ("name", "ilike", "Deco"),
        ],
        "limit": 1,
        "offset": 1,
    },
).json()
print("Partner ids (with limit/offset)", partners)

### 2.2: Product search

In [ ]:
product_var = requests.post(
    f"{url}/json/2/product.product/search_read",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "domain": [
            ("default_code", "=", "E-COM11"),
        ],
        "fields": ["id", "name", "default_code"],
    },
).json()
print("Product variant", product_var)

### 2.3: SO Creation

In [ ]:
so_data = {
    "partner_id": partner[0],
    "order_line": [
        # Magic number, please refer to Odoo documentation for more details (https://www.odoo.com/documentation/18.0/developer/reference/backend/orm.html#odoo.fields.Command)
        (0, 0, {"product_id": product_var[0]["id"]})
    ]
}

so = requests.post(
    f"{url}/json/2/sale.order/create",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "vals_list": [so_data],
    },
).json()
print("SO id", so)

### 2.4: Line Addition

In [ ]:
returned_value = requests.post(
    f"{url}/json/2/sale.order/write",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "ids": so,
        "vals": {
            "order_line": [
                # Magic number, please refer to Odoo documentation for more details (https://www.odoo.com/documentation/18.0/developer/reference/backend/orm.html#odoo.fields.Command)
                (0, 0, {
                    "product_id": product_var[0]["id"]
                })
            ]
        },
    },
).json()

print("Returned value", returned_value)

## 3 Send by mail

Send by mail button creates a wizard, we have to deal with that: create the wizard with the necessary informations (mail templates,...) then send the mail.

In [ ]:
# 'push' on send mail button, in user  interface this create the wizard. Here it returns a dictionnary with the infos required to create the wizard (with key 'context')

action_send_mail = requests.post(
    f"{url}/json/2/sale.order/action_quotation_send",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "ids": so,
    },
).json()

from pprint import pprint
print("Action send mail (returned wizard)")
pprint(action_send_mail)

In [ ]:
# Wizard creation, with context from previous request
action_context = action_send_mail.get("context")
wizard_args = [{}]

wizard_id = requests.post(
    f"{url}/json/2/mail.compose.message/create",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "vals_list": wizard_args,
        "context": action_context,
    },
).json()

print("Wizard id", wizard_id)


In [ ]:
# Send the mail
wizard_close = requests.post(
    f"{url}/json/2/mail.compose.message/action_send_mail",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "ids": wizard_id,
    },
).json()

print("Returned value", wizard_close)

## 4: Confirm SO

In [ ]:
returned_value = requests.post(
    f"{url}/json/2/sale.order/action_confirm",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "ids": so,
    },
).json()
print("Returned Value", returned_value)

## 5: Add log note

In [ ]:
returned_value = requests.post(
    f"{url}/json/2/sale.order/message_post",
    headers={
        # "X-Odoo-Database": f"{db},  # only when DOMAIN isn't enough
        "Authorization": f"bearer {api_key}",
    },
    json={
        "ids": so,
        "body": "This SO has been confirmed through API",
    },
).json()

print("Returned Value", returned_value)

## 2) Request limit on odoo online/SH
Can be limited to 1 requests/sec. Note that on odoo SH it looks like the request are only slowed down (a lot)